# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
symbol = 'MSFT'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Microsoft Corporation',
 'marketcap': 1792978520937,
 'week52high': 249.77,
 'week52low': 137.09,
 'week52change': 0.39189167677014786,
 'sharesOutstanding': 7726427321,
 'float': 0,
 'avg10Volume': 50410618,
 'avg30Volume': 34604052,
 'day200MovingAvg': 219.7,
 'day50MovingAvg': 222.57,
 'employees': 0,
 'ttmEPS': 6.81,
 'ttmDividendRate': 2.120391203153558,
 'dividendYield': 0.009157415663085415,
 'nextDividendDate': '2021-02-05',
 'exDividendDate': '2020-11-17',
 'nextEarningsDate': '2021-01-19',
 'peRatio': 35.2045911599039,
 'beta': 1.18612608227039,
 'maxChangePercent': 9.336154411483927,
 'year5ChangePercent': 3.6734886163866847,
 'year2ChangePercent': 1.294552621696434,
 'year1ChangePercent': 0.3804905637575061,
 'ytdChangePercent': 0.04322469939999738,
 'month6ChangePercent': 0.14376395632061587,
 'month3ChangePercent': 0.1538961034930425,
 'month1ChangePercent': 0.04436300667102856,
 'day30ChangePercent': 0.04426482658260576,
 'day5ChangePercent': 0.027793012

In [5]:
data['year1ChangePercent']

0.3804905637575061

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [59]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', '1-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [60]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_url_call = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url_call).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index = True
        )
        
final_dataframe

,Ticker,Price,1-Year Price Return,Number of Shares to Buy
0,A,123.170,0.471250,N/A
1,AAL,17.361,-0.362216,N/A
2,AAP,152.150,0.143840,N/A
3,AAPL,137.690,0.726643,N/A
4,ABBV,104.780,0.334186,N/A
...,...,...,...,...
95,CINF,89.350,-0.180171,N/A
96,CL,80.500,0.085578,N/A
97,CLX,215.060,0.371553,N/A
98,CMA,60.870,-0.000411,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [61]:
final_dataframe.sort_values('1-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe.head(10)
final_dataframe.reset_index(drop=True, inplace = True)
final_dataframe

,Ticker,Price,1-Year Price Return,Number of Shares to Buy
0,CARR,39.514,2.289030,N/A
1,ALB,173.680,1.085077,N/A
2,ALGN,542.890,1.049873,N/A
3,ABMD,357.470,0.879137,N/A
4,AMD,90.850,0.829904,N/A
5,CDNS,135.260,0.812046,N/A
6,AAPL,137.690,0.726643,N/A
7,AMAT,104.484,0.686578,N/A
8,APTV,139.430,0.603805,N/A
9,AMZN,3446.708,0.602799,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [62]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the value of your portfolio:')

    try:
        portfolio_size = abs(float(portfolio_size))
    except ValueError:
        print("That's not a number! \nPlease try again")
        portfolio_size = input('Enter the value of your portfolio:')
        portfolio_size = abs(float(portfolio_size)) # Value automatically converted to a positive
        
portfolio_input()
print(portfolio_size)

Enter the value of your portfolio: 1000000


1000000.0


In [63]:
position_size = portfolio_size/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
    
final_dataframe

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,1-Year Price Return,Number of Shares to Buy
0,CARR,39.514,2.289030,2530
1,ALB,173.680,1.085077,575
2,ALGN,542.890,1.049873,184
3,ABMD,357.470,0.879137,279
4,AMD,90.850,0.829904,1100
5,CDNS,135.260,0.812046,739
6,AAPL,137.690,0.726643,726
7,AMAT,104.484,0.686578,957
8,APTV,139.430,0.603805,717
9,AMZN,3446.708,0.602799,29


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [64]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    '1yr Price Return',
    '1yr Return Percentile',
    '6mo Price Return',
    '6mo Return Percentile',
    '3mo Price Return',
    '3mo Return Percentile',
    '1mo Price Return',
    '1mo Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_url_call = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url_call).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'n/a',
                    data[symbol]['stats']['year1ChangePercent'],
                    'n/a',
                    data[symbol]['stats']['month6ChangePercent'],
                    'n/a',
                    data[symbol]['stats']['month3ChangePercent'],
                    'n/a',
                    data[symbol]['stats']['month1ChangePercent'],
                    'n/a',
                    'n/a'
                ],
                index = hqm_columns
            ),
            ignore_index = True
        )

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,1yr Price Return,1yr Return Percentile,6mo Price Return,6mo Return Percentile,3mo Price Return,3mo Return Percentile,1mo Price Return,1mo Return Percentile,HQM Score
0,A,124.36,n/a,0.488943,n/a,0.2526,n/a,0.179862,n/a,0.0162668,n/a,n/a
1,AAL,17.73,n/a,-0.374334,n/a,0.559605,n/a,0.543739,n/a,0.0917714,n/a,n/a
2,AAP,151.75,n/a,0.145943,n/a,-0.00368152,n/a,0.0147243,n/a,-0.0546007,n/a,n/a
3,AAPL,135.41,n/a,0.746412,n/a,0.257089,n/a,0.222631,n/a,-0.00553661,n/a,n/a
4,ABBV,106.18,n/a,0.339162,n/a,0.112556,n/a,0.22206,n/a,-0.0337108,n/a,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,104.54,n/a,-0.0227717,n/a,0.131132,n/a,0.0966974,n/a,-0.0679672,n/a,n/a
501,ZBH,158.36,n/a,0.0473241,n/a,0.145992,n/a,0.168112,n/a,-0.00273899,n/a,n/a
502,ZBRA,405.90,n/a,0.640612,n/a,0.395913,n/a,0.381207,n/a,0.00941719,n/a,n/a
503,ZION,46.36,n/a,0.00729877,n/a,0.398043,n/a,0.385859,n/a,0.0163542,n/a,n/a


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [65]:
# There seems to be some 'None' values in the dataframe that could cause problems.

hqm_dataframe[hqm_dataframe.isna().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,1yr Price Return,1yr Return Percentile,6mo Price Return,6mo Return Percentile,3mo Price Return,3mo Return Percentile,1mo Price Return,1mo Return Percentile,HQM Score
118,CTL,11.00,n/a,None,n/a,None,n/a,None,n/a,None,n/a,n/a
165,ETFC,50.70,n/a,None,n/a,None,n/a,None,n/a,None,n/a,n/a
326,MYL,16.09,n/a,None,n/a,None,n/a,None,n/a,None,n/a,n/a
327,NBL,8.52,n/a,None,n/a,None,n/a,None,n/a,None,n/a,n/a


In [66]:
time_periods = [
                '1yr',
                '6mo',
                '3mo',
                '1mo'
]

# Replace 'None' with 0s

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col],
                                                       hqm_dataframe.loc[row, change_col])
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,1yr Price Return,1yr Return Percentile,6mo Price Return,6mo Return Percentile,3mo Price Return,3mo Return Percentile,1mo Price Return,1mo Return Percentile,HQM Score
0,A,124.36,n/a,0.488943,87.9208,0.2526,66.1386,0.179862,58.0198,0.0162668,68.5149,n/a
1,AAL,17.73,n/a,-0.374334,2.77228,0.559605,92.2772,0.543739,95.6436,0.0917714,91.2871,n/a
2,AAP,151.75,n/a,0.145943,56.8317,-0.00368152,16.4356,0.0147243,15.4455,-0.0546007,28.1188,n/a
3,AAPL,135.41,n/a,0.746412,96.2376,0.257089,67.3267,0.222631,65.5446,-0.00553661,55.4455,n/a
4,ABBV,106.18,n/a,0.339162,81.9802,0.112556,39.4059,0.22206,65.3465,-0.0337108,38.8119,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,104.54,n/a,-0.0227717,33.0693,0.131132,42.3762,0.0966974,35.6436,-0.0679672,19.0099,n/a
501,ZBH,158.36,n/a,0.0473241,43.3663,0.145992,45.5446,0.168112,56.2376,-0.00273899,57.4257,n/a
502,ZBRA,405.90,n/a,0.640612,94.4554,0.395913,82.5743,0.381207,85.7426,0.00941719,65.5446,n/a
503,ZION,46.36,n/a,0.00729877,39.0099,0.398043,83.3663,0.385859,85.9406,0.0163542,68.9109,n/a


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [67]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_precentiles = []
    for time_period in time_periods:
        momentum_precentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])    
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_precentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,1yr Price Return,1yr Return Percentile,6mo Price Return,6mo Return Percentile,3mo Price Return,3mo Return Percentile,1mo Price Return,1mo Return Percentile,HQM Score
0,A,124.36,n/a,0.488943,87.9208,0.2526,66.1386,0.179862,58.0198,0.0162668,68.5149,70.1485
1,AAL,17.73,n/a,-0.374334,2.77228,0.559605,92.2772,0.543739,95.6436,0.0917714,91.2871,70.495
2,AAP,151.75,n/a,0.145943,56.8317,-0.00368152,16.4356,0.0147243,15.4455,-0.0546007,28.1188,29.2079
3,AAPL,135.41,n/a,0.746412,96.2376,0.257089,67.3267,0.222631,65.5446,-0.00553661,55.4455,71.1386
4,ABBV,106.18,n/a,0.339162,81.9802,0.112556,39.4059,0.22206,65.3465,-0.0337108,38.8119,56.3861
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,104.54,n/a,-0.0227717,33.0693,0.131132,42.3762,0.0966974,35.6436,-0.0679672,19.0099,32.5248
501,ZBH,158.36,n/a,0.0473241,43.3663,0.145992,45.5446,0.168112,56.2376,-0.00273899,57.4257,50.6436
502,ZBRA,405.90,n/a,0.640612,94.4554,0.395913,82.5743,0.381207,85.7426,0.00941719,65.5446,82.0792
503,ZION,46.36,n/a,0.00729877,39.0099,0.398043,83.3663,0.385859,85.9406,0.0163542,68.9109,69.3069


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [68]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe.head(50)
hqm_dataframe.reset_index(inplace=True, drop=True)

In [69]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,1yr Price Return,1yr Return Percentile,6mo Price Return,6mo Return Percentile,3mo Price Return,3mo Return Percentile,1mo Price Return,1mo Return Percentile,HQM Score
0,ALB,174.370,n/a,1.07389,99.2079,1.02339,99.0099,0.783953,98.2178,0.104865,93.4653,97.4752
1,SIVB,465.570,n/a,0.837523,98.0198,0.99762,98.6139,0.529427,95.0495,0.131297,96.2376,96.9802
2,VIAC,54.310,n/a,0.484053,87.7228,0.916325,98.2178,0.741755,98.0198,0.304702,99.604,95.8911
3,DISCA,42.270,n/a,0.42627,85.5446,1.00379,98.8119,1.08579,99.2079,0.388834,100,95.8911
4,GM,53.340,n/a,0.548151,90.297,1.06935,99.2079,0.479298,92.0792,0.22109,98.8119,95.099
5,AMAT,103.460,n/a,0.70423,95.4455,0.513551,89.703,0.66361,97.4257,0.121974,95.2475,94.4554
6,FCX,28.360,n/a,1.49061,99.802,1.11802,99.4059,0.552484,96.0396,0.0356902,77.4257,93.1683
7,DISCK,36.526,n/a,0.272955,74.6535,0.857656,98.0198,0.928379,98.8119,0.338847,99.802,92.8218
8,LLY,207.636,n/a,0.534855,89.901,0.406225,83.7624,0.614395,97.0297,0.236949,99.0099,92.4257
9,IVZ,22.295,n/a,0.268615,73.6634,1.12907,99.604,0.594298,96.8317,0.182915,98.2178,92.0792


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [52]:
portfolio_input()

Enter the value of your portfolio: 1000000


In [70]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
    
hqm_dataframe

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,1yr Price Return,1yr Return Percentile,6mo Price Return,6mo Return Percentile,3mo Price Return,3mo Return Percentile,1mo Price Return,1mo Return Percentile,HQM Score
0,ALB,174.370,114,1.07389,99.2079,1.02339,99.0099,0.783953,98.2178,0.104865,93.4653,97.4752
1,SIVB,465.570,42,0.837523,98.0198,0.99762,98.6139,0.529427,95.0495,0.131297,96.2376,96.9802
2,VIAC,54.310,368,0.484053,87.7228,0.916325,98.2178,0.741755,98.0198,0.304702,99.604,95.8911
3,DISCA,42.270,473,0.42627,85.5446,1.00379,98.8119,1.08579,99.2079,0.388834,100,95.8911
4,GM,53.340,374,0.548151,90.297,1.06935,99.2079,0.479298,92.0792,0.22109,98.8119,95.099
5,AMAT,103.460,193,0.70423,95.4455,0.513551,89.703,0.66361,97.4257,0.121974,95.2475,94.4554
6,FCX,28.360,705,1.49061,99.802,1.11802,99.4059,0.552484,96.0396,0.0356902,77.4257,93.1683
7,DISCK,36.526,547,0.272955,74.6535,0.857656,98.0198,0.928379,98.8119,0.338847,99.802,92.8218
8,LLY,207.636,96,0.534855,89.901,0.406225,83.7624,0.614395,97.0297,0.236949,99.0099,92.4257
9,IVZ,22.295,897,0.268615,73.6634,1.12907,99.604,0.594298,96.8317,0.182915,98.2178,92.0792


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [99]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [100]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [101]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['1yr Price Return', percent_template],
    'E': ['1yr Return Percentile', integer_template],
    'F': ['6mo Price Return', percent_template],
    'G': ['6mo Return Percentile', integer_template],
    'H': ['3mo Price Return', percent_template],
    'I': ['3mo Return Percentile', integer_template],
    'J': ['1mo Price Return', percent_template],
    'K': ['1mo Return Percentile', integer_template],
    'L': ['HQM Score', integer_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [ ]:
writer.save()